In [1]:
import pandas as pd
import os
import glob
from time import sleep
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import numpy as np

## First, concatenate all CSVs & export

In [2]:
# collect all CSVs and combine into one DF
extension = 'csv'
result = glob.glob('2023/*.{}'.format(extension))
joined_df = pd.concat((pd.read_csv(f) for f in result)).drop_duplicates().reset_index(drop=True)

# filter out bad transactions
joined_df['salePrice_numeric'] = joined_df['Sale Price'].replace('[\$,]', '', regex=True).astype(float)
joined_df = joined_df[joined_df['salePrice_numeric'] >= 10000]
joined_df = joined_df[joined_df['Square Ft '] > 75]

# from this, create the price per SF column
joined_df['price_SF'] = joined_df['salePrice_numeric'] / joined_df['Square Ft '] 

# pare down the columns
joined_df = joined_df[[
    'Address',
    'Sale Date',
    'Sale Price',
    'salePrice_numeric',
    'price_SF',
    'Square Ft ',
    'Year  Built '
]]

joined_df = joined_df.rename(columns={
    'Square Ft ':'sq_ft',
    'Year  Built ':'yr_built',
    'Sale Date':'sale_date',
    'Sale Price':'sale_price'
})

# change salePrice_numeric data type
joined_df['salePrice_numeric'] = joined_df['salePrice_numeric'].astype(int)

# change data column
joined_df['sale_date'] = pd.to_datetime(joined_df['sale_date'])

# create unique ID column that we'll use to remove dupes before geocoding
joined_df['unique_ID'] = joined_df['Address'].str.replace(' ','') + '-' + joined_df['sale_date'].astype(str) + '-' + joined_df['salePrice_numeric'].astype(str)

# now remove dupes
joined_df = joined_df.drop_duplicates(subset='unique_ID')

# strip whitespace
joined_df['unique_ID'] = joined_df['unique_ID'].str.strip()

joined_df

,Address,sale_date,sale_price,salePrice_numeric,price_SF,sq_ft,yr_built,unique_ID
3,3530 PIEDMONT RD NE # F 11,2023-02-27,"$375,000.00",375000,245.579568,1527,1976.0,3530PIEDMONTRDNE#F11-2023-02-27-375000
11,968 WELCH ST SE,2023-02-28,"$127,000.00",127000,127.000000,1000,1952.0,968WELCHSTSE-2023-02-28-127000
14,845 SPRING ST NW 229,2023-02-28,"$588,500.00",588500,349.258160,1685,2002.0,845SPRINGSTNW229-2023-02-28-588500
21,606 WILLARD AVE SW,2023-02-28,"$199,000.00",199000,52.107882,3819,1930.0,606WILLARDAVESW-2023-02-28-199000
22,839 ROCHELLE DR SW,2023-02-28,"$425,000.00",425000,204.130644,2082,1946.0,839ROCHELLEDRSW-2023-02-28-425000
...,...,...,...,...,...,...,...,...
3365,589 ROCK SPRINGS RD NE,2023-03-01,"$651,000.00",651000,402.846535,1616,1939.0,589ROCKSPRINGSRDNE-2023-03-01-651000
3367,4153 ROSWELL RD NE,2023-03-01,"$1,100,000.00",1100000,721.784777,1524,0.0,4153ROSWELLRDNE-2023-03-01-1100000
3369,2516 FORREST WAY NE,2023-03-01,"$650,000.00",650000,417.201540,1558,1935.0,2516FORRESTWAYNE-2023-03-01-650000
3374,1390 EAST FORREST AVE,2023-03-01,"$211,000.00",211000,257.631258,819,1940.0,1390EASTFORRESTAVE-2023-03-01-211000


## Remove addresses that already have lat / long values

In [3]:
geocoded = pd.read_csv('GeocodedAddresses_2023.csv')

# change data column
geocoded['Sale Date'] = pd.to_datetime(geocoded['Sale Date'])

geocoded['unique_ID'] = geocoded['Address'].str.replace(' ','') + '-' + geocoded['Sale Date'].astype(str) + '-' + geocoded['salePrice_numeric'].astype(str)

# strip whitespace
geocoded['unique_ID'] = geocoded['unique_ID'].str.strip()

# pare down the dataframe
geocoded = geocoded[['unique_ID', 'lat', 'long']]

new_df = joined_df.merge(geocoded, how='left', left_on='unique_ID', right_on='unique_ID')

# now, only grab the rows that don't have lat / long values
new_df = new_df[new_df['lat'].isnull()]

new_df = new_df.drop(columns=['lat', 'long'])

new_df

/var/folders/yt/f4vdhy_56sg4y2gk8bwqbcvc0000gn/T/ipykernel_17441/1494719984.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  geocoded['Sale Date'] = pd.to_datetime(geocoded['Sale Date'])


,Address,sale_date,sale_price,salePrice_numeric,price_SF,sq_ft,yr_built,unique_ID
191,3740 WEST HARVARD AVE # R,2023-02-22,"$75,000.00",75000,31.250000,2400,0.0,3740WESTHARVARDAVE#R-2023-02-22-75000
579,1602 WOODCLIFF DR NE,2023-06-21,"$268,000.00",268000,236.958444,1131,1982.0,1602WOODCLIFFDRNE-2023-06-21-268000
580,11870 NORTH HICKORY TRL,2023-06-21,"$590,000.00",590000,207.673354,2841,1986.0,11870NORTHHICKORYTRL-2023-06-21-590000
581,783 ESSIE AVE SE,2023-06-13,"$395,000.00",395000,255.498060,1546,1920.0,783ESSIEAVESE-2023-06-13-395000
582,855 HIGHLAND BEND COVE,2023-06-12,"$975,000.00",975000,238.678091,4085,1996.0,855HIGHLANDBENDCOVE-2023-06-12-975000
...,...,...,...,...,...,...,...,...
1655,4760 HOLLY GRAPE DR,2023-03-01,"$462,200.00",462200,142.478422,3244,2022.0,4760HOLLYGRAPEDR-2023-03-01-462200
1656,7914 LAUREL CREEK DR,2023-03-01,"$549,900.00",549900,275.501002,1996,2018.0,7914LAURELCREEKDR-2023-03-01-549900
1658,365 WINDING RIVER RD D,2023-03-01,"$189,000.00",189000,134.232955,1408,1970.0,365WINDINGRIVERRDD-2023-03-01-189000
1659,10525 GRANDVIEW SQ,2023-03-01,"$1,270,000.00",1270000,315.999005,4019,2019.0,10525GRANDVIEWSQ-2023-03-01-1270000


## Baby...preparrrre to geocode!

In [4]:
# create the full address
new_df['full_address'] = new_df['Address'] + ' Fulton County, GA'

# set the 'space filler' that Google Maps uses on a browser, then read in address data
space_filler = '%20'
new_df['Address_URL'] = new_df['full_address'].str.replace(' ', space_filler)

# creates the url which will be used to return the response below
new_df['url'] = ['https://www.google.com/maps/search/' + i for i in new_df['Address_URL']]

# # only split up the batch if you have a big chunk to geocode
# df1, df2 = np.array_split(joined_df, 2)
# df1.to_csv('fultonGeocode_batch1.csv')
# df2.to_csv('fultonGeocode_batch2.csv')

new_df

,Address,sale_date,sale_price,salePrice_numeric,price_SF,sq_ft,yr_built,unique_ID,full_address,Address_URL,url
191,3740 WEST HARVARD AVE # R,2023-02-22,"$75,000.00",75000,31.250000,2400,0.0,3740WESTHARVARDAVE#R-2023-02-22-75000,"3740 WEST HARVARD AVE # R Fulton County, GA",3740%20WEST%20HARVARD%20AVE%20#%20R%20Fulton%2...,https://www.google.com/maps/search/3740%20WEST...
579,1602 WOODCLIFF DR NE,2023-06-21,"$268,000.00",268000,236.958444,1131,1982.0,1602WOODCLIFFDRNE-2023-06-21-268000,"1602 WOODCLIFF DR NE Fulton County, GA","1602%20WOODCLIFF%20DR%20NE%20Fulton%20County,%...",https://www.google.com/maps/search/1602%20WOOD...
580,11870 NORTH HICKORY TRL,2023-06-21,"$590,000.00",590000,207.673354,2841,1986.0,11870NORTHHICKORYTRL-2023-06-21-590000,"11870 NORTH HICKORY TRL Fulton County, GA",11870%20NORTH%20HICKORY%20TRL%20Fulton%20Count...,https://www.google.com/maps/search/11870%20NOR...
581,783 ESSIE AVE SE,2023-06-13,"$395,000.00",395000,255.498060,1546,1920.0,783ESSIEAVESE-2023-06-13-395000,"783 ESSIE AVE SE Fulton County, GA","783%20ESSIE%20AVE%20SE%20Fulton%20County,%20GA",https://www.google.com/maps/search/783%20ESSIE...
582,855 HIGHLAND BEND COVE,2023-06-12,"$975,000.00",975000,238.678091,4085,1996.0,855HIGHLANDBENDCOVE-2023-06-12-975000,"855 HIGHLAND BEND COVE Fulton County, GA",855%20HIGHLAND%20BEND%20COVE%20Fulton%20County...,https://www.google.com/maps/search/855%20HIGHL...
...,...,...,...,...,...,...,...,...,...,...,...
1655,4760 HOLLY GRAPE DR,2023-03-01,"$462,200.00",462200,142.478422,3244,2022.0,4760HOLLYGRAPEDR-2023-03-01-462200,"4760 HOLLY GRAPE DR Fulton County, GA","4760%20HOLLY%20GRAPE%20DR%20Fulton%20County,%20GA",https://www.google.com/maps/search/4760%20HOLL...
1656,7914 LAUREL CREEK DR,2023-03-01,"$549,900.00",549900,275.501002,1996,2018.0,7914LAURELCREEKDR-2023-03-01-549900,"7914 LAUREL CREEK DR Fulton County, GA","7914%20LAUREL%20CREEK%20DR%20Fulton%20County,%...",https://www.google.com/maps/search/7914%20LAUR...
1658,365 WINDING RIVER RD D,2023-03-01,"$189,000.00",189000,134.232955,1408,1970.0,365WINDINGRIVERRDD-2023-03-01-189000,"365 WINDING RIVER RD D Fulton County, GA",365%20WINDING%20RIVER%20RD%20D%20Fulton%20Coun...,https://www.google.com/maps/search/365%20WINDI...
1659,10525 GRANDVIEW SQ,2023-03-01,"$1,270,000.00",1270000,315.999005,4019,2019.0,10525GRANDVIEWSQ-2023-03-01-1270000,"10525 GRANDVIEW SQ Fulton County, GA","10525%20GRANDVIEW%20SQ%20Fulton%20County,%20GA",https://www.google.com/maps/search/10525%20GRA...


In [5]:
options = Options()
options.add_argument("--headless=new")
driver = webdriver.Chrome(options=options)

results = []

for ind in tqdm(new_df.index, colour='#00FFFF', desc='Geocoding Progress'):
    try:
        driver.get(new_df['url'][ind])
        sleep(3.7)
        url = driver.current_url
        results.append(url)
    except:
        results.append('error')

print('Geocoding complete!')

Error getting version of chromedriver 115. Retrying with chromedriver 114 (attempt 1/5)
Geocoding Progress: 100%|█████████████████| 1044/1044 [1:21:43<00:00,  4.70s/it]

Geocoding complete!


## Post-geocoding processing pt. 1

In [6]:
# extract lat / longs & put into new dataframe
lats = []
longs = []

# parse & split the 'results' list 
for item in range(len(results)):
    try:
        found = re.search('/@(.+?),17z', results[item]).group(1)
        lats.append(found.split(',')[0])
        longs.append(found.split(',')[1])
    except:
        lats.append('error')
        longs.append('error')
        
# now add the parsed & cleaned lat/longs as additional columns 
df = new_df.copy()
df['lat'] = lats
df['long'] = longs

## Post-geocoding processing pt. 2

In [7]:
# how many addresses will need to be manually geocoded?
manuals = df[df['lat'] == 'error'].shape[0]
total_addresses = df.shape[0]
perc = (manuals / total_addresses) * 100
print(f'Addresses to manually geocode: {manuals} out of {total_addresses}, or only {perc:.1f}% of the total!')

# export to CSV
df.to_csv('GeocodedAddresses_2023_July.csv')
print("fee-neeshed!")

Addresses to manually geocode: 50 out of 1044, or only 4.8% of the total!
fee-neeshed!
